<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [ ]:
from esper.widget import *
from esper.prelude import *
from esper.spark_util import *

import pyspark.sql.functions as func
from collections import defaultdict
import random
import pickle
import json
from datetime import datetime
from pytz import timezone
from pathlib import Path

In [ ]:
OVERWRITE = True

In [ ]:
face_id_to_male_prob = {}
for f in get_face_genders().select('face_id', 'male_probability').collect():
    face_id_to_male_prob[f['face_id']] = f['male_probability']

In [ ]:
face_id_to_info = {}
for f in get_faces().select(
    'id', 'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2',
    'video_id', 'channel_id', 'show_id', 'canonical_show_id', 'in_commercial'
).collect():
    face_id_to_info[f['id']] = (
        f['bbox_x1'], f['bbox_x2'], f['bbox_y1'], f['bbox_y2'],
        f['video_id'], f['channel_id'], f['show_id'], f['canonical_show_id'], f['in_commercial'],
        face_id_to_male_prob.get(f['id'], 0.5),
    )

In [ ]:
FACE_TABLE_PATH = 'widget_data/face_table.pkl'
if not OVERWRITE and os.path.exists(FACE_TABLE_PATH):
    raise Exception('File exists!')
    
with open(FACE_TABLE_PATH, 'wb') as f:
    pickle.dump(face_id_to_info, f) 

In [ ]:
FACE_TABLE_PATH = 'widget_data/face_table.pkl'
with open(FACE_TABLE_PATH, 'rb') as f:
    face_id_to_info = pickle.load(f) 

In [ ]:
count = 0
for k, v in face_id_to_info.items():
    count += 1